In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.url'):
    shutil.rmtree('./file/2.url')

os.makedirs('./file/2.url')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            url = input_.loc[a, 'Url']
            
            dict_param = {}
            list_param = url.split('?')[1].split('&') if '?' in url else []
            for param in list_param:
                dict_param[param.split('=')[0]] = param.split('=')[1]
            
            dict_param.pop('page', '')
            dict_param['sort'] = 'name+asc'
            
            url = url.split('?')[0] + '?'
            for key, value in dict_param.items():
                url += f'{key}={value}&'

            list_url, i = [], 1
            while True:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(f'{url}page={i}',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
    
                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            list_href = html.xpath('//ul[@id="serp-list"]/li/div/div[@class="desktop-product-results"]/descendant::h4/a/@href')
                            if list_href:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_url += [f'''https://www.1aauto.com{href.strip()}''' for href in list_href]

                # = = = = = = = = = = = = = = =
    
                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
    
                print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']} > Page {i}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
    
                # = = = = = = = = = = = = = = =
    
                i += 1
    
                # = = = = = = = = = = = = = = =
    
                if not html.xpath('//a[@class="pagination-button pagination-button-next"]'):
                    break
                    
            # = = = = = = = = = = = = = = =
    
            if not list_url:
                raise
        
            # = = = = = = = = = = = = = = =

            output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                                   'Url': list_url})
            
            output = output.sort_values(by=['No'],
                                        ascending=[True],
                                        ignore_index=True)
            
            output.to_excel(f'''./file/2.url/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/url_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.url/{file}',
              f'''./file/2.url/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：107

[尝试次数：1] - 9.https://www.1aauto.com/auto-7-parts/b/24 > Page 1
[剩余数量：77] - [当前时间：14:26:57]

[尝试次数：1] - 18.https://www.1aauto.com/carrand-parts/b/194 > Page 1
[剩余数量：76] - [当前时间：14:26:57]

[尝试次数：1] - 5.https://www.1aauto.com/armor-all-parts/b/202 > Page 1
[剩余数量：75] - [当前时间：14:26:57]

[尝试次数：1] - 19.https://www.1aauto.com/catco-parts/b/263 > Page 1
[剩余数量：75] - [当前时间：14:26:57]

[尝试次数：1] - 23.https://www.1aauto.com/dea-parts/b/51 > Page 1
[剩余数量：74] - [当前时间：14:26:58]

[尝试次数：1] - 25.https://www.1aauto.com/depo-parts/b/100 > Page 1
[剩余数量：73] - [当前时间：14:26:58]

[尝试次数：1] - 8.https://www.1aauto.com/audi-parts/b/49 > Page 1
[剩余数量：73] - [当前时间：14:26:58]

[尝试次数：1] - 16.https://www.1aauto.com/cardone-reman-parts/b/253 > Page 1
[剩余数量：72] - [当前时间：14:26:58]

[尝试次数：1] - 2.https://www.1aauto.com/accu-form-parts/b/260 > Page 1
[剩余数量：72] - [当前时间：14:26:58]

[尝试次数：1] - 4.https://www.1aauto.com/acdelco-and-gm-parts/b/62 > Page 1
[剩余数量：72] - [当前时间：14:26:59]

[尝试次数：1] - 13.https://www.1aauto.com/beck-arnl

Progress: 100%|██████████████████████████| 107/107 [00:00<00:00, 6402.51it/s]

Done ~
